#### ***Importación de las librerias utiles***

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from skimage import feature
from PIL import Image 
from tqdm import tqdm
import os
from skimage.io import imread_collection
import tensorflow as tf
from sklearn.cluster import KMeans
import pywt
from scipy.stats import gennorm
from sklearn import preprocessing

#### ***Función para leer imágenes***

In [3]:
def convert2image(data_list, size):
    img_data = np.reshape(np.array(data_list), (size,size))
    return  img_data

#### ***Selección y limpieza de datos a clasificar (Camisetas y pantalones)***

In [4]:
data_clothes = pd.read_csv('dataset2/images.csv')
data_clothes['image'] = data_clothes['image'] + '.jpg'
data_clothes = data_clothes[['image', 'label']]
data_clothes_to_remove = ['T-Shirt', 'Pants', 'Shoes', 'Shorts', 'Dress']
data_clothes = data_clothes[data_clothes['label'].isin(data_clothes_to_remove)]

data_clothes = data_clothes[0:2000]

"""
T-Shirt  -> 4
Pants    -> 1
Shoes    -> 2
Dress    -> 0
Shorts   -> 3  
"""
label_encoder1 = preprocessing.LabelEncoder()
data_clothes["label"] = label_encoder1.fit_transform(data_clothes["label"])

# Classes and their counts within the dataset
print(data_clothes['label'].value_counts())

4    774
1    509
0    273
2    228
3    216
Name: label, dtype: int64


#### ***Funciones para almacenar los datos seleccionados a tratar***

In [5]:
def save_train_image():
    k = 1
    for i in tqdm(data_clothes.index.tolist()[0:1600]):
        img_path = data_clothes['image'][i]
        original_path = 'dataset2/images_original/' + img_path        
        image =  Image.open(original_path)       
        image = tf.image.resize(image, (224,224), preserve_aspect_ratio=False, antialias=False)  
        image = tf.image.rgb_to_grayscale(image)
        image  = tf.keras.utils.array_to_img(image) 
        new_path = 'dataset2/images_train/train' + str(k)  + '.jpg'
        k += 1
        if os.path.exists(new_path): 
            break
        else:
            image = image.save(new_path)

def save_test_image():
    k = 1
    for i in tqdm(data_clothes.index.tolist()[1601:2000]):
        img_path = data_clothes['image'][i]
        original_path = 'dataset2/images_original/' + img_path        
        image =  Image.open(original_path)   
        image = tf.image.resize(image, (224,224), preserve_aspect_ratio=False, antialias=False)  
        image = tf.image.rgb_to_grayscale(image)
        image  = tf.keras.utils.array_to_img(image) 
        new_path = 'dataset2/images_test/test' + str(k) + '.jpg'
        k += 1
        if os.path.exists(new_path): 
            break
        else:
            image = image.save(new_path)   

            
def save_new_csv(path, data):
    if os.path.exists(path): 
        pass
    else:
        data.to_csv(path, index=False)

#### ***Almacenamiento de los datos a tratar en carpetas a parte separando por nombres (train y test) y se crea un CSV con los datos a tratar***

In [6]:
save_train_image()
save_test_image()
save_new_csv('dataset2/CSV/clothes.csv', data_clothes)

100%|██████████| 399/399 [00:52<00:00,  7.65it/s]
